In [1]:
import csv
import cv2
import numpy as np
%matplotlib inline

lines = []
with open ('./data/driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        lines.append(line)
    
images = []
measurements = []

for line in lines:
    source_path = line[0]
    if source_path == 'center':
        continue
    filename = source_path.split('/')[-1]
    current_path = './data/IMG/' + filename
    image = cv2.imread(current_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    images.append(image)
    measurement = float(line[3])
    measurements.append(measurement)
    
# Augment data with fliping images and taking opposite sign of the steering measuremnts
augmented_images, augmented_measurements = [], []
for image, measurement in zip(images, measurements):
    augmented_images.append(image)
    augmented_measurements.append(measurement)
    augmented_images.append(cv2.flip(image, 1))
    augmented_measurements.append(measurement*(-1.0))

X_train = np.array(augmented_images)
y_train = np.array(augmented_measurements)

In [ ]:
from keras.models import Sequential
from keras.layers import Lambda, Conv2D, MaxPooling2D, Dropout, Dense, Flatten, Cropping2D
from keras.optimizers import Adam
from keras.layers.advanced_activations import ELU

dropout = 0.5

model = Sequential()

# Normalize & Cropping
model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160, 320, 3)))
model.add(Cropping2D(cropping=((50, 20), (0, 0))))

# Add 5 convolutional layers
model.add(Conv2D(24, kernel_size=(5, 5), activation='elu', strides=(2,2)))
model.add(Dropout(dropout))
model.add(Conv2D(36, kernel_size=(5, 5), activation='elu', strides=(2,2)))
model.add(Dropout(dropout))
model.add(Conv2D(48, kernel_size=(5, 5), activation='elu', strides=(2,2)))
model.add(Dropout(dropout))
model.add(Conv2D(64, kernel_size=(3, 3), activation='elu'))
model.add(Dropout(dropout))
model.add(Conv2D(64, kernel_size=(3, 3), activation='elu'))
model.add(Dropout(dropout))

# Add a flatten layer
model.add(Flatten())

# Add 4 fully connected layers
model.add(Dense(1164, activation='elu'))
model.add(Dropout(dropout))
model.add(Dense(100, activation='elu'))
model.add(Dropout(dropout))
model.add(Dense(50, activation='elu'))
model.add(Dropout(dropout))
model.add(Dense(10, activation='elu'))
model.add(Dropout(dropout))

# Add a fully connected output layers
model.add(Dense(1))

# Compile and train the model
optimizer = Adam(lr=0.001)
model.compile(loss='mse', optimizer=optimizer)
model.fit(X_train, y_train, validation_split=0.2, shuffle=True, epochs=5)

# Print model summary
print(model.summary)

# Save model
model.save('model.h5')

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/models.py:942: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 12857 samples, validate on 3215 samples
Epoch 1/5
12857/12857 [==============================] - 53s 4ms/step - loss: 1.5709 - val_loss: 0.0247
Epoch 2/5
12857/12857 [==============================] - 51s 4ms/step - loss: 0.2467 - val_loss: 0.0175
Epoch 3/5
12857/12857 [==============================] - 51s 4ms/step - loss: 0.0936 - val_loss: 0.0168
Epoch 4/5
12857/12857 [==============================] - 51s 4ms/step - loss: 0.0539 - val_loss: 0.0169
Epoch 5/5
 9440/12857 [=====================>........] - ETA: 12s - loss: 0.0389